# AgentScope 新手教程：三国狼人杀（Notebook版）

这个教程把 `main_cn.py` 的案例改造成适合初学者的 Jupyter 体验。

你将学会：
- AgentScope 在这个项目里负责什么
- 如何逐步检查环境与依赖
- 如何运行完整游戏并观察多智能体协作

## 0. 先决条件

1. 已安装依赖（建议在项目根目录执行）：
   - `pip install -r code/chapter6/AgentScopeDemo/requirements.txt`
2. 已配置环境变量：
   - `DASHSCOPE_API_KEY`
3. 笔记本建议从 `code/chapter6/AgentScopeDemo` 目录运行

In [ ]:
import os
import sys
from pathlib import Path

# 确保当前目录是 AgentScopeDemo，避免相对导入失败
project_dir = Path.cwd()
target_dir = Path('/root/workspace/hello-agents/code/chapter6/AgentScopeDemo')
if project_dir != target_dir:
    os.chdir(target_dir)

if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

print('当前工作目录:', Path.cwd())
print('DASHSCOPE_API_KEY 已设置:', 'DASHSCOPE_API_KEY' in os.environ)

## 1. 先理解架构（小白版）

这个项目分成两层：

- **框架层（AgentScope 提供）**
  - `ReActAgent`：智能体对象
  - `DashScopeChatModel`：模型调用
  - `MsgHub`：多人消息广播
  - `fanout_pipeline`：并发收集投票/行动

- **业务层（本项目自己写）**
  - 狼人杀规则
  - 各阶段流程（夜晚、白天、投票）
  - 胜负判断与主持人播报

In [ ]:
# 关键依赖导入：这里就能看出我们是直接使用 AgentScope
from agentscope.agent import ReActAgent
from agentscope.model import DashScopeChatModel
from agentscope.pipeline import MsgHub, fanout_pipeline

print('AgentScope 核心模块导入成功')

## 2. 快速烟雾测试（可选，但很推荐）

先创建一个最小智能体，确认 API Key 和模型调用都正常。

In [ ]:
import asyncio

async def smoke_test():
    if 'DASHSCOPE_API_KEY' not in os.environ:
        print('❌ 未检测到 DASHSCOPE_API_KEY，跳过测试')
        return

    agent = ReActAgent(
        name='SmokeTester',
        sys_prompt='你是一个简洁的助手，请只返回一句话。',
        model=DashScopeChatModel(
            model_name='qwen-max',
            api_key=os.environ['DASHSCOPE_API_KEY'],
            enable_thinking=False,
        ),
    )

    resp = await agent('请用一句话介绍 AgentScope。')
    print('✅ 烟雾测试完成')
    print(resp)

await smoke_test()

## 3. 运行完整案例（复用原脚本核心类）

为了让新手更快上手，这里直接复用 `main_cn.py` 中已经写好的 `ThreeKingdomsWerewolfGame`。

你可以先跑通，再回到源码逐个阅读：
1. `setup_game()`：创建角色和阵营
2. `werewolf_phase()`：狼人协商并击杀
3. `day_phase()`：白天讨论与投票
4. `run_game()`：主循环串联所有阶段

In [ ]:
from main_cn import ThreeKingdomsWerewolfGame

async def run_full_game():
    if 'DASHSCOPE_API_KEY' not in os.environ:
        print('❌ 请先设置 DASHSCOPE_API_KEY')
        return

    print('🎮 启动完整三国狼人杀...')
    game = ThreeKingdomsWerewolfGame()
    await game.run_game()

# 首次运行可能较慢（多智能体多轮对话）
await run_full_game()

## 4. 新手实验建议

可以从这三个小实验开始：

1. **减少讨论轮数**：修改 `utils_cn.py` 里的 `MAX_DISCUSSION_ROUND`，观察速度变化
2. **调整角色配置**：修改 `game_roles.py` 的人数与身份比例，观察胜率变化
3. **优化提示词**：修改 `prompt_cn.py` 的角色提示，看发言风格如何改变

完成后你就能理解：
- AgentScope 负责通信与调度
- 业务规则由你自己定义